### Overnight Returns

[Overnight Returns and Firm-Specific Investor Sentiment](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554010)

> **Abstract**: We explore the possibility that overnight rturns can serve as a measure of firm-specific investor sentimatne by analyzing whether they exhibit characteristics expected of a sentiment measure. First, we document short-term persistence in share demand of sentiment-influenced in share demand of sentiment-influenced retain investors. Second, we dind that short-term persistence is stonger for harder-to-value firms, consistent with evidence that sentiment palys a larger role when there is less objective data available for valuation. Thirrd, we show that stocks with high(low) overnight returns underperform (outperform) over the longer-term, consistent with evidence of temporary sentiment-driven mispricing.

> ** P 2 I**: The recent work of Berkman, Koch, Tuttle, and Zhang(2012) suggests that a stock's overnight (close-to_open) return can serve as a measure of firm-level sentiment.

> **p3, I**: Specifically, Berkman et al. (2012) finds that attention-generating events ( high absolute returns or storn net buying by retail investors) on one day leads to higher demand by individual investors, concentrated near the open of the next day...This creates temporary price pressur at the open, resulting in evalated overnight returns that are reversed during the next trading day.

> **p 3 I**: We conduct three setns of analysis. **In the first we test for short run persistence in overnight returns** The basis for expecting this from a measure of sentiment is the evidence in BArber et al. (2009) that the order imbalances of retail investors, who are the investors most likely to exhibit sentiment, persist for periods extending over several weeks...In the third analysis we examine whether stocks with high overnight returns underperform those with low overnight returns over the long term.

In [1]:
# Imports
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.factors import Returns
from zipline.pipeline.factors import DailyReturns

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import time

from quantopian.pipeline.experimental import QTradableStocksUS

### Pipeline construction using QTradableStocksUS

In [2]:
def make_pipeline():
    average_day_dv_200 = AverageDollarVolume(window_length = 200)
    market_cap = Fundamentals.market_cap.latest
    price_open = USEquityPricing.open.latest
    price_close = USEquityPricing.close.latest
    volume = USEquityPricing.volume.latest
    sector = Sector()
    daily_returns = DailyReturns(inputs = [USEquityPricing.close])
    
    # Ranked Returns
    returns = Returns(window_length=252)
    ranked_retuns = returns.rank()
    
    # New Factors
    # create a factor of 1 year returns, demeaned by sector, rank, zscore
    factor = (
    Returns(window_length=252).\
    demean(groupby = Sector()).\
    rank().\
    zscore()
    )
    
    # use the newly created factor as an input into SimpleMovingAverage, with window length = 5
    factor_smoothed = (SimpleMovingAverage(inputs = [factor], window_length=5).\
                      rank().\
                      zscore()
                      )
    
    return Pipeline(
        columns = {
            'AverageDollarVolume': average_day_dv_200,
            'MarketCap': market_cap,
            'close_price':price_close,
            'open_price':price_open,
            'volume':volume,
            'sector':sector,
            '1yrReturns': factor,
            '5dAvgReturns': factor_smoothed,
            'ranked_returns':ranked_retuns,
            '1dReturns':daily_returns
        },
        screen = QTradableStocksUS()
    )

In [3]:
# pipeline is run over this time range and outputs a dataframe indexed by asset name:
start_date = '2017'
end_date = '2019-08-03'

QTU_pipeline2 = run_pipeline(make_pipeline(), start_date, end_date, chunksize=252)

In [4]:
# lets see what stocks had the most gains ytd through August 2nd
leading_ytd_stocks = QTU_pipeline2.sort_values(by=['ranked_returns'], ascending = False)
leading_ytd_stocks.filter(like='2019-08-02', axis = 0).head(25)

1dReturns  1yrReturns  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])   0.018846    1.729817   
                          Equity(42749 [ENPH])   0.069627    1.725988   
                          Equity(32726 [EHTH])  -0.012531    1.720882   
                          Equity(32215 [APPS])   0.027599    1.718330   
                          Equity(50735 [AYX])    0.120985    1.711948   
                          Equity(50477 [IIPR])  -0.036067    1.712586   
                          Equity(48628 [NVTA])   0.011165    1.714501   
                          Equity(50288 [TTD])    0.004899    1.708757   
                          Equity(50411 [RARX])  -0.032021    1.711310   
                          Equity(31341 [ZIOP])  -0.025937    1.705566   
                          Equity(51314 [MDB])    0.004120    1.700460   
                          Equity(50449 [INSG])  -0.048170    1.697908   
                          Equity(50758 [OKTA])   0.017274    1.695993   
                          Equity(4549 [LSCC])   -0.001552    1.693440   
                          Equity(49060 [SHOP])   0.074229    1.692164   
                          Equity(45758 [VCYT])  -0.046610    1.701099   
                          Equity(49460 [NVCR])   0.032223    1.699184   
                          Equity(51961 [SMAR])   0.001202    1.685782   
                          Equity(51832 [ZS])     0.011867    1.685144   
                          Equity(44991 [NSTG])  -0.126945    1.694078   
                          Equity(47321 [CDNA])   0.036314    1.692802   
                          Equity(50077 [TWLO])  -0.008914    1.680676   
                          Equity(48254 [WK])     0.013572    1.671104   
                          Equity(47779 [CYBR])  -0.003242    1.670465   
                          Equity(45429 [FATE])  -0.030413    1.688973   

                                                5dAvgReturns  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])      1.729248   
                          Equity(42749 [ENPH])      1.720529   
                          Equity(32726 [EHTH])      1.723643   
                          Equity(32215 [APPS])      1.719906   
                          Equity(50735 [AYX])       1.707450   
                          Equity(50477 [IIPR])      1.716169   
                          Equity(48628 [NVTA])      1.713055   
                          Equity(50288 [TTD])       1.711187   
                          Equity(50411 [RARX])      1.713678   
                          Equity(31341 [ZIOP])      1.703090   
                          Equity(51314 [MDB])       1.696862   
                          Equity(50449 [INSG])      1.704958   
                          Equity(50758 [OKTA])      1.694993   
                          Equity(4549 [LSCC])       1.678800   
                          Equity(49060 [SHOP])      1.662607   
                          Equity(45758 [VCYT])      1.706204   
                          Equity(49460 [NVCR])      1.695616   
                          Equity(51961 [SMAR])      1.685028   
                          Equity(51832 [ZS])        1.681291   
                          Equity(44991 [NSTG])      1.706827   
                          Equity(47321 [CDNA])      1.702467   
                          Equity(50077 [TWLO])      1.683160   
                          Equity(48254 [WK])        1.676932   
                          Equity(47779 [CYBR])      1.673818   
                          Equity(45429 [FATE])      1.699353   

                                                AverageDollarVolume  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])         1.541528e+07   
                          Equity(42749 [ENPH])         2.810533e+07   
                          Equity(32726 [EHTH])         2.175493e+07   
                          Equity(32215 [APPS])         3.525766e+06   
                          Equity(50735 [AYX])          6.546713e+07   
                          Equity(50477 [IIPR])         2.4

In [5]:
# lets see what stocks had the highest daily returns on Aug 2nd
leading_daily_stocks = QTU_pipeline2.sort_values(by=['ranked_returns'], ascending = False)
leading_daily_stocks.filter(like='2019-08-02', axis = 0).head(25)

1dReturns  1yrReturns  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])   0.018846    1.729817   
                          Equity(42749 [ENPH])   0.069627    1.725988   
                          Equity(32726 [EHTH])  -0.012531    1.720882   
                          Equity(32215 [APPS])   0.027599    1.718330   
                          Equity(50735 [AYX])    0.120985    1.711948   
                          Equity(50477 [IIPR])  -0.036067    1.712586   
                          Equity(48628 [NVTA])   0.011165    1.714501   
                          Equity(50288 [TTD])    0.004899    1.708757   
                          Equity(50411 [RARX])  -0.032021    1.711310   
                          Equity(31341 [ZIOP])  -0.025937    1.705566   
                          Equity(51314 [MDB])    0.004120    1.700460   
                          Equity(50449 [INSG])  -0.048170    1.697908   
                          Equity(50758 [OKTA])   0.017274    1.695993   
                          Equity(4549 [LSCC])   -0.001552    1.693440   
                          Equity(49060 [SHOP])   0.074229    1.692164   
                          Equity(45758 [VCYT])  -0.046610    1.701099   
                          Equity(49460 [NVCR])   0.032223    1.699184   
                          Equity(51961 [SMAR])   0.001202    1.685782   
                          Equity(51832 [ZS])     0.011867    1.685144   
                          Equity(44991 [NSTG])  -0.126945    1.694078   
                          Equity(47321 [CDNA])   0.036314    1.692802   
                          Equity(50077 [TWLO])  -0.008914    1.680676   
                          Equity(48254 [WK])     0.013572    1.671104   
                          Equity(47779 [CYBR])  -0.003242    1.670465   
                          Equity(45429 [FATE])  -0.030413    1.688973   

                                                5dAvgReturns  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])      1.729248   
                          Equity(42749 [ENPH])      1.720529   
                          Equity(32726 [EHTH])      1.723643   
                          Equity(32215 [APPS])      1.719906   
                          Equity(50735 [AYX])       1.707450   
                          Equity(50477 [IIPR])      1.716169   
                          Equity(48628 [NVTA])      1.713055   
                          Equity(50288 [TTD])       1.711187   
                          Equity(50411 [RARX])      1.713678   
                          Equity(31341 [ZIOP])      1.703090   
                          Equity(51314 [MDB])       1.696862   
                          Equity(50449 [INSG])      1.704958   
                          Equity(50758 [OKTA])      1.694993   
                          Equity(4549 [LSCC])       1.678800   
                          Equity(49060 [SHOP])      1.662607   
                          Equity(45758 [VCYT])      1.706204   
                          Equity(49460 [NVCR])      1.695616   
                          Equity(51961 [SMAR])      1.685028   
                          Equity(51832 [ZS])        1.681291   
                          Equity(44991 [NSTG])      1.706827   
                          Equity(47321 [CDNA])      1.702467   
                          Equity(50077 [TWLO])      1.683160   
                          Equity(48254 [WK])        1.676932   
                          Equity(47779 [CYBR])      1.673818   
                          Equity(45429 [FATE])      1.699353   

                                                AverageDollarVolume  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])         1.541528e+07   
                          Equity(42749 [ENPH])         2.810533e+07   
                          Equity(32726 [EHTH])         2.175493e+07   
                          Equity(32215 [APPS])         3.525766e+06   
                          Equity(50735 [AYX])          6.546713e+07   
                          Equity(50477 [IIPR])         2.4

### Create individual custom dataframes

In [6]:
df_momentum = QTU_pipeline2[['1yrReturns', '5dAvgReturns']]
df_price = QTU_pipeline2[['close_price']]
df_returns = QTU_pipeline2[['1dReturns']]

df_price = df_price.close_price.unstack().fillna(0)
df_returns = df_returns['1dReturns'].unstack().fillna(0)

### Walk through the "Returns" class
we'll walk through how the `Returns` class works, because we'll create a new class that inherits from `Returns` in order to calculated a customized return.

### Returns inherits from CustomFactor
The zipline package has a class [zipline.pipeline.factors.Returns](https://www.zipline.io/appendix.html?highlight=returns#zipline.pipeline.factors.Returns) which inherits from class [zipline.pipeline.CustomFactor](https://www.zipline.io/appendix.html?highlight=custom%20factor#zipline.pipeline.CustomFactor).  The [source code for Returns is here](https://www.zipline.io/_modules/zipline/pipeline/factors/basic.html#Returns), and the [source code for CustomFactor is here.](https://www.zipline.io/_modules/zipline/pipeline/factors/factor.html#CustomFactor) 

**Please open the links to the documentation and source code and follow along with our notes about the code**

### Inputs Variable
The CustomFactor class takes the `inputs` as a parameter of the constructor for the class, otherwise it looks for a class-level variable named `inputs`. `inputs` takes a list of BoundColumn instances. These help us choose what kind of price-volume data to use as input. The `Returns` class sets this to

```
inputs = [USEquityPricing.close]
```

### USEquityPricing Class
The class [USEquityPricing](https://www.zipline.io/appendix.html?highlight=usequitypricing#zipline.pipeline.data.USEquityPricing) has a couple BoundColumn instances that we can choose from:

close = USEquityPricing.close

high = USEquityPricing.high

low = USEquityPricing.low

open = USEquityPricing.open

volume = USEquityPricing.volume

### Quiz 1
If we wish to calculate close to open rturns, which columns from USEquityPricing do you think we'll want ot put into the list and set as `inputs`?

### Answer 1
USEquityPricing.close & USEquityPricing.open

### window_length variable
The CustomFactor class takes `window_length` (an integer) as a constructor parameter, otherwise it looks for a class-level variable named `window_length`. If we chose a `window_length = 2` then ths means that we'll be passing two days' worth of data (two rows) into the `compute` function.

### Quiz 2
What window length would you choose if you were calculating daily close to open returns? Assume we have daily data:

### Answer 2
window_length of 2 to have 2 days of data

### Compute Function
The function definition of the `Returns` class includes the compute function
```
    def compute(self, today, assets, out, close):
        out[:] = (close[-1] - close[0]) / close[0]
```

* `today`: this is handled by parent classes; it has the datetime for the "today" row for the given subset of data. We wont use it for this function implementation.
* `assets`: this is handled by parent classes: it has the column header names fo the "out" and "close". We won't use it for this fucntion implementation.
* `out`: this points to a numpy array that will store the result of our compute. It stores our "return" value of the `compute` function instead of explicitly returning a variable.
* `*input`: a tuple of numpy arrays that contain input data that we'll use t compute a signal. In the 'Returns' definition of `compute`, the input is a single value `close` but we can list more if we need additional columns of data to comupte a return

if we set teh `window_length = 2` , then the `compute` fucntion gets two rows worth of data from `close`. The index 1 value is the most recent value, adn the index value 0 is the earliest in time. Recall that in Python, the -1 index is the same as getting the highest indexed value, so with a numpy array of just length two, -1 gives use the value at index 1.

So the line of code is calculating the one-day return using the close price and storing that into the `out` variable.

$Return = \frac{close_1 - close_0}{close_0} $

### Quiz 3
Given a numpy array for open prices called `open` and a numpy array for close prices called `close`, what code would you write to get the most recent open price? Assum that you have 2 days of data.

### Answer 3
open[1] or open[-1] (because python lets us do a reverse indexing) are valid answers. Use the -1 index allows us to get the most recent price ( the very last indes of the numpy array) regardless of the window length, you may prefer to use -1 to make your code easier to maintain or modify.

### Close to Open Returns (Overnight Returns)
The close-to-open return is the change in price between when the market closed on one day and when it opened on the next. So it's

$
CloseToOpen = \frac{open_1 - close_0}{close_0}
$

We'll now creae a class `CTO` that inherits from `Return`, and override  the `compute` function

### Quiz 4
Create a customized class `CloseToOpenReturns` that inherit from the Returns class. Define that compute function to calculate overnight returns.

In [13]:
class CloseToOpenReturns(Returns):
    """
    the following function crates the return between today's opening price vs. last nights closing price
    """
    # Set window_length (we're calculating daily returns)
    window_length = 2
    
    # set inputs
    inputs = [USEquityPricing.open, USEquityPricing.close]
    
    # The compute method is passed the current day, the assets list, a pre-allocated out vector and the
    # factor's items in the list 'inputs'
    def compute(self, today, assets, out, opens, closes):
        # calculate close-to-open return and save into out[:]
        out[:] = (opens[-1] - closes[0]) / closes[0]

In [20]:
CloseToOpenReturns()

CloseToOpenReturns([EquityPricing<US>.open, EquityPricing<US>.close], 2)

### Trailing overnight returns
The cumulative overnight returns over a week may be predictive of future returns; hence it's kind of a momentum signal.

$
TrailingOvernightReturns = \sum_{1}^{Days}CloseToOpen_1
$
Where $Days$ could be 5 if we are looking at a weekly window.

So we want ot take the `CloseToOpenReturns` as our input into another class, `TrailingOvernightReturns` which also inherits from `Returns`.

### mask
Note that we're going to creae another class that inherits from `Returns`. Recall that `Returns` inherits from [CustomFactor](), which has a `mask` parameter for its constructor. The `mask` parameter takes in a `Filter` object, which determines which stock series get passed to the `compute` function. Note that when we used `AverageDollarVolume` and stored its output  in the variable `universe`, this `universe` variable is of type `Filter`.

### Quiz 5
if you wanted to creae an object of the type `CloseToOpen` and also defind the object so that it only computes returns on the set of stocks in universe that we selected eariler in this notebook, what code would you write?

### Answer 5
WE could instantiate a `CloseToOpen` object wth `CloseToOpenRetuns(mask=universe), and this would only calculate the close to open returns for the stocks defined in ur universe variable.

### numpy.nansum
Numpy has a `namsum` function that treat NaN (not a number) as zeros. Note that by default if we give numpy.nansum a 2D numpy array, it will calculate a single sum across all rows and columns. For our purposes, we want to compute a sum over 5 days (5 rows), and each column has daily close to open returns for a single stock. It helps to think of the matrix (2D numpy array) as a nested list of lists. This makes it easier to  decide whether to set `axis = 0` or `axis = 1`.


```
    temp = 
    [
        [stock1day1, stock2day1]
        [stock1day2, stock2day2]
        ...
    ]
```

if we look at the outermost list, each element is a list that represents one day's worth of data. If we used `np.nansum(tmp, axis = 0)`,this would sum across the days for each stock. If we think of this as a 2D matrix, setting `axis = 0` is like calculating a sum for each column.

If we set `axis = 0` this applies `nansum` to the outermost list (axis 0), so that we end up with:

```
[
    sum_of_stock_1, sum_of_stock_2
]
```

Alternatively, if we set `axis = 1`, this applies `nansum` to the lists nested inside the outermost list. Each of these nested lists represent data for a single day, for all stocks, so that we get:

```
    [
    sum_of_day_1,
    sum_of_day_2
    ]
```

### Example using numpy.nansum

In [14]:
tmp = np.array([
    [1,2,3],
    [np.nan, np.nan, np.nan],
    [1,1,1]
])

np.nansum(tmp), np.nansum(tmp, axis = 0), np.nansum(tmp, axis=1)

(9.0, array([ 2.,  3.,  4.]), array([ 6.,  0.,  3.]))

### Quiz 6
For our purposes, we want a sum for each stock series. Which axis do you think we should use?

### Answer 6
WE want to set axis = 0 so that we have a sum for each stock (each column).

### Quiz 7
Create a class TrailingOvernightReturns that inherits from Returns and takes the cumulative weekly sum of overnight reutrns

In [31]:
universe = AverageDollarVolume(window_length=120).top(500) 

In [32]:
class TrailingOvernightReturns(Returns):
    """
    Sum of trailing close-to-open returns; we expect sentiment persistence at short horizons, 
    so we look at the 5-day (i.e. 1 week) window
    """
    
    # choose a window_length to calculate a weekly return
    window_length = 5
    
    # set inputs to a list containing the daily close to open returns
    # filter the close to open returns by our stock universe
    
    inputs = [CloseToOpenReturns(mask=universe)]
    
    def compute(self, today, assets, out, close_to_open):
        # calcualte the sum of close_to_open
        # choose the axis so that ther is a sum for each stock (each column)
        # treat NaN as zeros
        out[:] = np.nansum(close_to_open, axis = 0)

In [33]:
TrailingOvernightReturns()

TrailingOvernightReturns([CloseToOpenReturns(...)], 5)

# Quiz 8
Create a factor by instantiating the TrailingOvernightReturns class that you just defined. Deman by sector, rank, and convert to a zscore.

In [34]:
def make_pipeline():
    average_day_dv_200 = AverageDollarVolume(window_length = 200)
    market_cap = Fundamentals.market_cap.latest
    price = USEquityPricing.close.latest
    volume = USEquityPricing.volume.latest
    sector = Sector()
    daily_returns = DailyReturns(inputs = [USEquityPricing.close])
    
    # Ranked Returns
    returns = Returns(window_length = 252)
    ranked_returns = returns.rank()
    
    # New Factors
    # create a factor of 1 year returns, demeaned by sector, rank, zscore
    factor = (
    Returns(window_length=252).\
    demean(groupby=Sector()).\
    rank().
    zscore()
    )
    
    # use the newly created factor as a input into SimpleMovingAverage, with windowlength = 5
    factor_smoothed = (SimpleMovingAverage(inputs = [factor], window_length=5).\
                      rank().
                      zscore()
                      )
    
    # create an overnight_returns_factor variable
    overnight_returns_factor = (
    TrailingOvernightReturns().\
        demean(groupby = Sector()).\
        rank().\
        zscore()
    )
    
    return Pipeline(
        columns = {
            'AverageDollarVolume':average_day_dv_200,
            'MarketCap':market_cap,
            'Price':price,
            'Volume':volume,
            'Sector':sector,
            '1yrReturns':factor,
            '5dAvgReturns':factor_smoothed,
            'ranked_return':ranked_returns,
            '1dReturns':daily_returns,
            'o_n_sentiment': overnight_returns_factor
        },
        screen = QTradableStocksUS()
    )

In [35]:
make_pipeline()

In [36]:
# pipeline is run over this time range and outputs a data frame indexed by asset name:
start_date = '2017'
end_date = '2019-08-02'

QTU_pipeline3 = run_pipeline(make_pipeline(), start_date, end_date, chunksize=252)

In [37]:
QTU_pipeline3

1dReturns  1yrReturns  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])      -0.008552   -1.663162   
                          Equity(24 [AAPL])     -0.007624    0.242636   
                          Equity(31 [ABAX])     -0.001893    0.731982   
                          Equity(41 [ARCB])     -0.015957    0.571808   
                          Equity(52 [ABM])       0.001717    1.027217   
                          Equity(53 [ABMD])     -0.008446    1.407291   
                          Equity(62 [ABT])       0.002348    0.527013   
                          Equity(64 [GOLD])     -0.024405    1.425616   
                          Equity(67 [ADSK])     -0.013068    0.628819   
                          Equity(76 [TAP])      -0.007141   -0.091286   
                          Equity(84 [ACET])      0.002281    0.448284   
                          Equity(110 [RAMP])    -0.004085    0.945773   
                          Equity(114 [ADBE])    -0.007137    0.229062   
                          Equity(122 [ADI])     -0.013049    1.038755   
                          Equity(128 [ADM])      0.006170    0.805281   
                          Equity(154 [AEM])     -0.032258    0.225669   
                          Equity(161 [AEP])     -0.005370   -0.201914   
                          Equity(166 [AES])     -0.006835    0.459821   
                          Equity(168 [AET])     -0.003455    1.160243   
                          Equity(185 [AFL])     -0.003293    0.063459   
                          Equity(197 [AGCO])    -0.003101    0.372269   
                          Equity(216 [HES])     -0.009539   -1.352995   
                          Equity(239 [AIG])     -0.002748   -0.360052   
                          Equity(247 [AIN])     -0.008556    0.924055   
                          Equity(253 [AIR])     -0.013425    0.368875   
                          Equity(266 [AJG])      0.002314    0.560948   
                          Equity(270 [AKRX])     0.032395   -0.863650   
                          Equity(289 [MATX])    -0.007009   -1.105947   
                          Equity(300 [ALK])      0.002146   -0.234492   
                          Equity(301 [ALKS])     0.004698   -0.180196   
...                                                   ...         ...   
2019-08-02 00:00:00+00:00 Equity(52211 [TLRY])  -0.012022    1.633450   
                          Equity(52233 [BE])    -0.021053   -1.638556   
                          Equity(52241 [TENB])  -0.011976   -0.923782   
                          Equity(52242 [FOCS])  -0.058444   -1.159274   
                          Equity(52244 [BRY])   -0.078571    0.039249   
                          Equity(52288 [CWK])   -0.006051         NaN   
                          Equity(52291 [SONO])  -0.021237         NaN   
                          Equity(52361 [BPR])   -0.001294         NaN   
                          Equity(52407 [FTDR])  -0.008762         NaN   
                          Equity(52408 [GTX])   -0.046610         NaN   
                          Equity(52424 [ELAN])   0.003336         NaN   
                          Equity(52425 [FTCH])  -0.004478         NaN   
                          Equity(52427 [EB])    -0.003953         NaN   
                          Equity(52446 [UPWK])  -0.010341         NaN   
                          Equity(52450 [SVMK])  -0.025354         NaN   
                          Equity(52491 [GH])     0.005107         NaN   
                          Equity(52494 [ESTC])  -0.013455         NaN   
                          Equity(52508 [ALLO])  -0.021627         NaN   
                          Equity(52511 [LTHM])  -0.034161         NaN   
                          Equity(52517 [PLAN])  -0.011591         NaN   
                          Equity(52525 [REZI])  -0.030753         NaN   
                          Equity(52529 [ACA])   -0.044800         NaN   
                          Equity(52537 [STNE])   0.015424         NaN   
                          Equity(